In [8]:
import torch
import torch.nn.functional as F
from flash_attn.flash_attn_interface import _flash_attn_forward
from flash_attn import flash_attn_func
import torch

In [2]:
import torch
torch.__version__

'2.8.0+cu128'

In [3]:
head_num = 16
dim = 128
seq_len = 100
chunk_size = 5
batch_size = 1

In [17]:
q = torch.randn(batch_size, head_num, seq_len, dim).cuda().to(torch.bfloat16)
k = torch.randn(batch_size, head_num, seq_len, dim).cuda().to(torch.bfloat16)
v = torch.randn(batch_size, head_num, seq_len, dim).cuda().to(torch.bfloat16)

In [28]:
out_dot = torch.nn.functional.scaled_dot_product_attention(q, k, v)
out_dot.shape

torch.Size([1, 16, 100, 128])

In [23]:
q_ = q.transpose(1, 2)
k_ = k.transpose(1, 2)
v_ = v.transpose(1, 2)
v_.shape

torch.Size([1, 100, 16, 128])

In [27]:
out_fa = flash_attn_func(q_, k_, v_, )
out_fa.shape

torch.Size([1, 100, 16, 128])

In [29]:
assert torch.allclose(out_fa, out_dot.transpose(1, 2), atol=0.125, rtol=0)

In [30]:
q_chunks = q_.chunk(chunk_size, dim = 1)
k_chunks = k_.chunk(chunk_size, dim = 1)
v_chunks = v_.chunk(chunk_size, dim = 1)

In [36]:
Q_block = q_chunks[0]
softmax_scale = Q_block.shape[-1] ** (-0.5)

In [50]:
out, lse = None, None

def merge_attention(a, lse_a, b, lse_b):
    max_lse = torch.maximum(lse_a, lse_b)
    lse_a = torch.exp(lse_a - max_lse)
    lse_b = torch.exp(lse_b - max_lse)
    out = ((a * lse_a[..., None] + b * lse_b[..., None]) / (lse_a + lse_b)[..., None])
    return out, torch.log(lse_a + lse_b) + max_lse

for i in range(len(k_chunks)):
    
    outputs = _flash_attn_forward(
        Q_block, 
        k_chunks[i], 
        v_chunks[i], 
        dropout_p=0, 
        softmax_scale=softmax_scale, 
        causal=False,
        window_size_left=-1,
        window_size_right=-1,
        softcap=0.0,
        alibi_slopes=None,
        return_softmax=False,
    )
    block_out, block_lse, _, _ = outputs
    block_lse = block_lse.transpose(1, 2)
    
    if out is None:
        out = block_out
        lse = block_lse
    else:
        out, lse = merge_attention(out, lse, block_out, block_lse)

In [55]:
out_fa[:,:out.shape[1]].argmax(-1)

tensor([[[ 91,  51,  40,  81,  51,  52,  71,  27,  75,  74,  90,  67,  88,  77,
           60,  91],
         [ 91, 120,  33,   7,  51,  91, 109,  55,   3, 122,  53,  67,  69,  19,
           60,  82],
         [ 51,  62,  40,  11, 113,  91,  85,   4, 107,  24,  58,   9,  90,  77,
            8,  10],
         [ 91,   3,  26,  80,  11,  91,  54,  99,  32,  95, 127,  30,  88,  19,
           56, 123],
         [ 91,  64,  46,  75,  62,  76,  27,  23, 111,  74,  58, 116,  29,  19,
           56,  85],
         [ 58,  76, 119,  90,  68, 103,  40,  92, 124, 102,  25,  67,  15,  19,
           60, 122],
         [ 91,  51,  61,  74,  51,  91, 108,  36, 119, 118,  91, 126, 109,  19,
          103, 122],
         [ 58,  64, 122,  82,   8,  45,  12,  20,   0,  84, 114, 102,  88,  19,
           93,  91],
         [ 81,  93,  40,  18,  51, 117,  71,  75,   0, 119,  29,  99,  76,   3,
           55, 114],
         [ 91, 120,  13,   2, 105,  41,  79,  36,  98,  84,  25,  25,  27,  19,
           

In [56]:
out.argmax(-1)

tensor([[[ 91,  51,  40,  81,  51,  52,  71,  27,  75,  74,  90,  67,  88,  77,
           60,  91],
         [ 91, 120,  33,   7,  51,  91, 109,  55,   3, 122,  53,  67,  69,  19,
           60,  82],
         [ 51,  62,  40,  11, 113,  91,  85,   4, 107,  24,  58,   9,  90,  77,
            8,  10],
         [ 91,   3,  26,  80,  11,  91,  54,  99,  32,  95, 127,  30,  88,  19,
           56, 123],
         [ 91,  64,  46,  75,  62,  76,  27,  23, 111,  74,  58, 116,  29,  19,
           56,  85],
         [ 58,  76, 119,  90,  68, 103,  40,  92, 124, 102,  25,  67,  15,  19,
           60, 122],
         [ 91,  51,  61,  74,  51,  91, 108,  36, 119, 118,  91, 126, 109,  19,
          103, 122],
         [ 58,  64, 122,  82,   8,  45,  12,  20,   0,  84, 114, 102,  88,  19,
           93,  91],
         [ 81,  93,  40,  18,  51, 117,  71,  75,   0, 119,  29,  99,  76,   3,
           55, 114],
         [ 91, 120,  13,   2, 105,  41,  79,  36,  98,  84,  25,  25,  27,  19,
           